In [1]:
import numpy as np
import pyvista as pv

from matplotlib import pyplot as plt

import nbimporter
from Design_Visualizer import read_config, Visualizer_Config, gen_voxel

In [2]:
gaussian_row = np.array([1, 3, 1])/5

def filter_along_2(array, filter_rad, filter_):
    # Iterating through slices
    for slice_ in range(array.shape[0]):
        for i in range(array.shape[1]):
            for j in range(filter_rad, array.shape[2] - filter_rad):
                array[slice_][i][j] = np.dot(array[slice_, i, j-filter_rad : j + filter_rad + 1], filter_)
                 
def filter_along_1(array, filter_rad, filter_):
    # Iterating through slices
    for slice_ in range(array.shape[0]):
        for i in range(array.shape[2]):
            for j in range(filter_rad, array.shape[1] - filter_rad):
                array[slice_][j][i] = np.dot(array[slice_, j-filter_rad : j + filter_rad + 1, i], filter_)
                
def filter_along_0(array, filter_rad, filter_):
    # Iterating through slices
    for slice_ in range(filter_rad, array.shape[0] - filter_rad):
        for i in range(array.shape[1]):
            for j in range(array.shape[2]):
                array[slice_][i][j] = np.dot(array[slice_ - filter_rad : slice_ + filter_rad + 1, i, j], filter_)

In [47]:
path = 'C:\\Users\\mihai\\Desktop\\Unik\\Optimization\\3D\\Data\\Arm_4_2.txt'
config = read_config(path)

In [48]:
def dens_tensor(config):
    density_tensor = np.zeros((config.ey, config.ez, config.ex))
    
    for ez_ in range(config.ez):
        for ex_ in range(config.ex):
            for ey_ in range(config.ey):
                elem_index = (config.ex*config.ey)*ez_ + config.ey * ex_ + (config.ey - ey_ - 1)
                
                density_tensor[ey_][ez_][ex_] = config.densities[elem_index]
                
    return density_tensor

In [53]:
def visualize_dt(density_tensor):
    ez, ey, ex = density_tensor.shape
    
    x_range = np.arange(ex+1, dtype = np.float32)
    y_range = np.arange(ey+1, dtype = np.float32)
    z_range = np.arange(ez+1, dtype = np.float32)
    
    x, y, z = np.meshgrid(x_range, y_range, z_range, indexing = 'xy')
    grid_ = pv.StructuredGrid(x, y, z)
    
    hided = []

    for ez_ in range(ez):
        for ex_ in range(ex):
            for ey_ in range(ey):
                elem_index_hide = (ex*ey)*ez_ + ey * ex_ + ey_
            
                if (density_tensor[ez_, ey_, ex_] < 0.1):
                    hided.append(elem_index_hide)          
                    
    grid_.hide_cells(hided, inplace = True)
    
    plotter = pv.Plotter()
    plotter.add_mesh(grid_, show_edges = False, show_vertices = False, point_size = 0, culling = False)
    plotter.show()
    #plotter.export_gltf('new.gltf')

In [54]:
dt = dens_tensor(config)

In [55]:
m_coef = 5
mult = np.ones((m_coef, m_coef, m_coef))
multiplied = np.kron(dt, mult)

filter_arr = [5, 10, 10, 15, 20, 15, 10, 10, 5]
filter_row = np.array(filter_arr)/sum(filter_arr)
filter_rad = 4

filter_along_2(multiplied, filter_rad, filter_row)
filter_along_1(multiplied, filter_rad, filter_row)
filter_along_0(multiplied, filter_rad, filter_row)

multiplied[multiplied < 0.35] = 0.0

In [56]:
visualize_dt(multiplied)

Widget(value="<iframe src='http://localhost:58880/index.html?ui=P_0x19e6f222100_10&reconnect=auto' style='widt…